In [1]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [9]:
# get data
%run preprocess.ipynb
X_train, y_train, X_val, y_val, X_test, y_test = get_processed_data(test_size=0.1, val_size=0.7)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(19821, 58)
(11894, 58)
(7929, 58)
targets: [[-0.14581653]
 [-0.15441734]
 [ 0.11220795]
 ...
 [ 0.09500632]
 [-0.25023043]
 [-0.1888206 ]]
data: [[-1.60772590e+00 -4.74451259e-01  1.78983723e-03 ...  5.47157435e-01
  -1.19274310e+00  2.67809642e-01]
 [ 1.23048186e+00  9.19437671e+00 -7.39034343e-02 ...  2.44637285e-01
  -1.56994907e+00 -8.70560322e-02]
 [ 7.57447233e-01 -3.51335557e-01  3.25676620e-03 ...  1.23781665e+00
  -2.21638357e-01  1.07797467e+00]
 ...
 [ 1.23048186e+00 -7.27050716e-01  2.79063632e-02 ...  1.23781665e+00
  -2.21638357e-01  1.07797467e+00]
 [ 2.84412606e-01  3.28703405e-02 -9.31336282e-03 ... -3.05226384e-01
  -1.13121305e+00 -6.47252789e-01]
 [-1.88622020e-01 -6.59124812e-01  1.32395104e-02 ... -2.69076177e-01
   8.37748634e-01 -6.89658121e-01]]
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFr

In [7]:
# load data without preprocessing
from ucimlrepo import fetch_ucirepo

# fetch dataset
online_news_popularity = fetch_ucirepo(id=332)

# data (as pandas dataframes)
X_raw = online_news_popularity.data.features
y_raw = online_news_popularity.data.targets

# metadata
print(online_news_popularity.metadata)
# variable information
print(online_news_popularity.variables)


{'uci_id': 332, 'name': 'Online News Popularity', 'repository_url': 'https://archive.ics.uci.edu/dataset/332/online+news+popularity', 'data_url': 'https://archive.ics.uci.edu/static/public/332/data.csv', 'abstract': 'This dataset summarizes a heterogeneous set of features about articles published by Mashable in a period of two years. The goal is to predict the number of shares in social networks (popularity).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 39797, 'num_features': 58, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': [' shares'], 'index_col': ['url'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Thu Feb 15 2024', 'dataset_doi': '10.24432/C5NS3V', 'creators': ['Kelwin Fernandes', 'Pedro Vinagre', 'Paulo Cortez', 'Pedro Sernadela'], 'intro_paper': {'title': 'A Proactive Intelligent Decision Support System for Predicti

In [21]:
# split unpreprocessed data into training, validation and test sets
X_train_raw, y_train_raw, X_val_raw, y_val_raw, X_test_raw, y_test_raw = split(X_raw, y_raw,test_size=0.1, val_size=0.7)

In [10]:
# create a random forest regressor model
rf = RandomForestRegressor(n_estimators=10, random_state=42, verbose=2) # n_estimators is the number of trees in the forest
rf.fit(X_train, y_train)
rf

/usr/local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.9s finished


RandomForestRegressor(n_estimators=10, random_state=42, verbose=2)

In [ ]:
rf = RandomForestRegressor(n_estimators=7, random_state=42, verbose=2) # n_estimators is the number of trees in the forest
rf.fit(X_raw, y_raw)
rf

In [3]:
%run evaluate_model.ipynb

In [11]:
# Caclulate performance using the validation set
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = rf.predict(X_val)
evaluate_model(y_val, y_pred)
y_pred = rf.predict(X_train)
evaluate_model(y_train, y_pred)


r2 score: -0.16903341049403875
mean absolute error: 0.30855069628537735
mean squared error: 1.25832712363728
r2 score: 0.7815258088855723
mean absolute error: 0.1130398652000845
mean squared error: 0.19855970326296912


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


(0.7815258088855723, 0.1130398652000845, 0.19855970326296912)

In [12]:
from sklearn.model_selection import GridSearchCV
import joblib

In [20]:
# tune hyperparameters
def grid_search_tunning(n_estimators, criterion, max_depth, min_samples_split, X_train, y_train, X_val, y_val):
    best_mse = float('inf')
    best_params = {}
    best_estimator = None
    for n in n_estimators:
        for c in criterion:
            for d in max_depth:
                for s in min_samples_split:
                    rf = RandomForestRegressor(n_estimators=n, criterion=c, max_depth=d, min_samples_split=s, random_state=42)
                    rf.fit(X_train, y_train)
                    y_pred = rf.predict(X_val)
                    mse = mean_squared_error(y_val, y_pred)
                    if mse < best_mse:
                        best_mse = mse
                        best_params = {'n_estimators': n, 'criterion': c, 'max_depth': d, 'min_samples_split': s}
                        best_estimator = rf
    return best_mse, best_params, best_estimator

In [24]:
best_mse, best_params, best_estimator = grid_search_tunning(n_estimators=[100],
                    criterion=["squared_error"],
                    max_depth=[2, 3, 4, 5, 6],
                    min_samples_split=[2],
                    X_train=X_train, y_train=y_train,
                    X_val=X_val,
                    y_val=y_val)

/usr/local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using r

In [25]:
best_mse

1.1462537944161044

In [26]:
best_params

{'n_estimators': 100,
 'criterion': 'squared_error',
 'max_depth': 4,
 'min_samples_split': 2}

In [28]:
y_pred = best_estimator.predict(X_val)
evaluate_model(y_val, y_pred)

r2 score: -0.0649130559187221
mean absolute error: 0.26506045967086533
mean squared error: 1.1462537944161044


(-0.0649130559187221, 0.26506045967086533, 1.1462537944161044)